In [209]:
from selenium import webdriver
from bs4 import BeautifulSoup
from PIL import ImageGrab
import requests
import os
import subprocess
import time
from datetime import datetime
from selenium.common import exceptions
import pandas as pd
import re
import random
import string
import pyautogui
from PIL import Image
from selenium.webdriver.common.keys import Keys

In [229]:
class Ecexl(object):
    
    def __init__(self,data,name =datetime.today()):
       
        self._to_csv(data, str(name))
    
    # 문제가 있음...
    def _to_csv(self,data,name):
        self.dataframe = pd.DataFrame(data,columns=['name','url','year'])
        #rint(self.dataframe)
        #column = ['name','url','year']\
        self.dataframe.to_csv(name+".csv",encoding='euc_kr', mode='a')
        

        

class Text(object):
    
    def __init__(self,file_name):
        self.file_name = file_name
        
        
    def _write_text(self,text):
        print(type(text))
        with open(self.file_name,'a',encoding='utf8') as file:
            file.write(text+"\n")
        
        

#Stting class
class Stting(object):

    def __init__(self):
        self.__options = webdriver.ChromeOptions()
        self.__options.add_argument('headless')
        self.__options.add_argument('window-size=1920x1080')
        self.__options.add_argument('--disable-gpu')
        self.__options.add_argument("lang=ko_KR")  # 한국어!
        self.__options.add_argument(
            "user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36")
        self.__driver = webdriver.Chrome('C:/Users/khk37/Downloads/chromedriver_win32/chromedriver')

    def get_driver(self):
        return self.__driver


class Crawling(object):

    def __init__(self):
        self.header = {
            "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_5)\AppleWebKit 537.36 (KHTML, like Gecko) Chrome",
            "Accept": "text/html,application/xhtml+xml,application/xml;\
                        q=0.9,imgwebp,*/*;q=0.8"}

        self.__driver = Stting().get_driver()
    
    
    def parsing_html(self, source):
        soup = BeautifulSoup(source, "lxml")
        return soup 
        

    def parsing(self, url):
        html = requests.get(url, headers=self.header)
        # 파싱
        soup = BeautifulSoup(html.content, "lxml")
        return soup

    def Crawling_run_naver(self, start_date, end_date, keyword, sort):
        # 해당 사이트 주소 리스트
        self.href = []
        self.company_name = []
        self.name_commpy = []
        self.select_user_company = []
        self.start_date = start_date
        self.end_date = end_date
        self.keyword = keyword
        self.page = 0

        # 접근 오류 해결하기 위함.
        requests.packages.urllib3.disable_warnings()
        page = 1
        s_from = self.start_date.replace(".", "")
        e_to = self.end_date.replace(".", "")
        # sort 0 : 관련도,  1 : 최신순 , 2 : 오래된

        max_page = (10 - 1) * 10 + 1

        while page <= max_page:

            url = "https://search.naver.com/search.naver?where=news&query=" + keyword + "&sort=" + str(
                sort) + "&ds=" + self.start_date + "&de=" + self.end_date + "&nso=so%3Ar%2Cp%3Afrom" + s_from + "to" + e_to + "%2Ca%3A&start=" + str(
                page)

            soup = self.parsing(url)

            # ._sp_each_url
            # ._sp_each_title
            new_title = soup.select("._sp_each_url")

            company_names = soup.select("._sp_each_source")

            years = soup.select(".txt_inline")
            
            
            # 뉴스 기사 주소 수집
            for urls, name in zip(new_title, company_names):
            #for urls, name, year in zip(new_title, company_names, years):
                self.href.append(urls['href'])
                try:

                   # pattern = '\d+.(\d+).(\d+).'  # 정규표현식
                    #r = re.compile(pattern)
                    #match = r.search(year.text).group(0)
                    
                    self.name_commpy.append(name.text.replace("언론사 선정", ""))
                    
                    #Ecexl({name.text.replace("언론사 선정", ""), urls['href'], match}, 'justTT')


                except Exception as e:
                    print(e)
            # continue

            # 네이버 페이지 전환 규칙
            page += 10

        self.collect_comment()

    def collect_comment(self):
        for news in self.href:
            self.__driver.implicitly_wait(3)
            self.__driver.get(news)

            # 최초 더보기 버튼 클릭
            self.__driver.find_element_by_css_selector(".u_cbox_btn_view_comment").click()
            self.__driver.implicitly_wait(3)

           
            
            print(self.name_commpy)
            # 더보기 버튼 계속 누르기.
            try:
                
                while True:
                    self.__driver.find_element_by_css_selector(".u_cbox_btn_more").click()
               
                    #pyautogui.screenshot('C:/Users/khk37/OneDrive/Desktop/Test/'+self.name_commpy[self.page]+''.join(random.choice(string.ascii_uppercase + string.digits)for  _ in range(12))+'.png')
                   
                    
                    #self.__driver.execute_script("window.scrollTo(0,2500);")
                    
                    
                    #time.sleep(1.5)
                
                
               

            except exceptions.ElementNotVisibleException as e:  # 페이지 끝
                pass

            except Exception as e:  # 다른 예외 발생시 확인
                print("에러 :  ", e)
            
            #document.body.scrollHeight
            #스크롤 끝으로 올림.
            self.__driver.execute_script("window.scrollTo(0, 0);")
            
           
            soup = self.parsing_html(self.__driver.page_source)
            comment_list = soup.find_all("span", {"class": "u_cbox_contents"})
            
            last_height = self.__driver.execute_script("return document.body.scrollHeight")
            
            elem = self.__driver.find_element_by_tag_name("body") 
            
            down = 0 
             
            number = 1
            
            collect_text = Text('collect_text.txt')
            
            #while True:
             #       png_name ='C:/Users/khk37/OneDrive/Desktop/Test/'+self.name_commpy[self.page]+''.join(random.choice(string.ascii_uppercase + string.digits)for  _ in range(12))+str(number)+'.png'
              #      number +=1
                    
                    #self.__driver.execute_script("window.scrollTo(0, document.body.scrollHeight//100);")
               #     elem.send_keys(Keys.PAGE_DOWN)
                #    self.__driver.save_screenshot(png_name)
                    
                    
                 #   im = Image.open(png_name)
                    # 가로 시작점, 세로 시작점, 가로 범위, 세로 범위
                  #  im = im.crop((int(5),int(5),int(580), int(1200)))
                   # im.save(png_name)
                    
                    #down +=1
                    #time.sleep(1)
            
                    #new_height =  self.__driver.execute_script("return document.body.scrollHeight")

                    #if new_height == last_height:
                    #   break
            
                    #last_height = new_height
                #element = self.__driver.find_element_by_css_selector('.u_cbox_area')
                   
            #location = element.location;
            #size = element.size;
                    
            #x = location['x'];
            #y = location['y'];
            #width = location['x']+size['width'];
            #height = location['y']+size['height'];
                    
            #print(x,y,width,height)

                    
            self.page +=1
            
            
            for comment in comment_list:
                  collect_text._write_text(comment.text) 
             
            #while True:
             #   self.__driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
              #  time.sleep(3)
            
            #a = soup.find_all("div", {"class": "u_cbox_area"})
            #a = self.__driver.find_element_by_css_selector('.u_cbox_area')
            #print(len(a))
            #print(a)
           
        # for idx,el in enumerate(a):
            #        el.save_screenshot(str(idx)+".png")
                    
            #for comment in comment_list:
             #   print(comment.text)


In [231]:
test = Crawling()
test.Crawling_run_naver('2019.7.00','2019.9.16','조국','0')

['뉴시스', '프레시안', '연합뉴스', '채널A', '연합뉴스', '경향신문', '중앙일보', '연합뉴스', '뉴스1', 'KBS', '동아일보', '프레시안', '경향신문', '중앙일보', '중앙일보', '연합뉴스', '뉴스1', '파이낸셜뉴스', 'KBS', '뉴스1', '조선일보', '중앙일보', '노컷뉴스', 'JTBC', '경향신문', '뉴스1', '프레시안', 'MBC', '세계일보', '서울신문', '아시아경제', '동아일보', 'MBC', '청년일보', '비즈니스워치', '오마이뉴스', '연합뉴스', '연합뉴스TV', '오마이뉴스', '전자신문', '조선일보', '시사오늘', '헤럴드경제', '세계일보', '파이낸셜뉴스', '중앙일보', 'MBC', '뉴스1', '중앙일보', '중앙일보', '아시아경제', '프레시안', '연합뉴스TV', '서울신문', '세계일보', '뉴시스', '동아일보', 'MBN', '조선일보', '시사위크', '헤럴드경제', '아시아경제', '조선일보', 'MBN', '뉴스1', '매일경제', '스카이데일리', '데일리안', '채널A', '국민일보', '노컷뉴스', '노컷뉴스', 'SBS CNBC', '뷰스앤뉴스', '중앙일보', '뉴시스', '뉴스핌', '파이낸셜뉴스', 'JTBC', '팍스넷뉴스', '노컷뉴스', '중앙일보', 'YTN', '한겨레', '뉴스1', '연합뉴스', 'MBN', '매일일보', '프레시안', '세계일보', 'tbs 교통방송', '프레시안', '한국일보', '스카이데일리', '시사위크', '조선일보', '경향신문', '아시아경제', '뉴스1', 'JTBC']
에러 :   Message: element not interactable
  (Session info: chrome=76.0.3809.132)

<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'st

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":".u_cbox_btn_view_comment"}
  (Session info: chrome=76.0.3809.132)


In [ ]:
print(''.join(random.choice(string.ascii_uppercase + string.digits)for  _ in range(12))+'.png')